# ECG 5000

In [26]:
# read the test data from the txt file 

import pandas as pd
import numpy as np

def read_data(filename):
    data = pd.read_csv(filename, sep=',', header=None)
    data = data.values
    return data

def remove_leading_spaces(input_file, output_file):
    lines = []
    with open(input_file, 'r') as f:
        for line in f:
            # strip the line of leading spaces
            line = line.lstrip()
            # replace the more than one space with a "," comma
            line = line.replace('  ', ',')
            lines.append(line)
        
    with open(output_file, 'w') as f:
        for line in lines:
            f.write(line)
                
                
                
input_file = 'ECG5000_TRAIN.txt'
output_file = 'ECG5000_TRAIN.csv'

test_input_file = 'ECG5000_TEST.txt'
test_output_file = 'ECG5000_TEST.csv'

remove_leading_spaces(input_file, output_file)
remove_leading_spaces(test_input_file, test_output_file)


In [28]:
# read the data
data = read_data(output_file)

test_data = read_data(test_output_file)

# print the shape of the data
print(data.shape)

(500, 141)
(141,)
